要运行此操作，请按“*Runtime*”，然后按“*运行*” ** free ** tesla Tesla T4 Google COLAB实例！
<div class =“ Align-Center”>
<a href =“ https://unsloth.ai/”> <img src =“ https://github.com/unslothai/unslothai/unsloth/raw/raw/main/main/mains/unsloth%20new%20new%20logo.png.png”
<a href =“ https://discord.gg/unsloth”> <img src =“ https://github.com/unslothai/unslothai/unsloth/raw/main/main/main/images/images/discord button.png button.png
<a href =“ https://docs.unsloth.ai/”> <img src =“ https://github.com/unslothai/unslothai/unsloth/unsloth/main/main/main/mains/images/documentation%20green%20green%20breen%20button.png?png?raw=raw=true width width” <i>在<a href =“ https://github.com/unslothai/unsloth”> github </a> </i>⭐中
</div>

要在您自己的计算机上安装不塞，请按照我们的github页面上的安装说明[here](https://docs.unsloth.ai/get-started/installing-+-updating)上的安装说明。

您将学习如何做[data prep](#Data)，如何[train](#Train)，如何[run the model](#Inference)，＆xx_markDown_link_xx


### 消息

Unsploth现在支持文本对语音（TTS）模型。阅读我们的xx_markDown_link_xx。

阅读我们的** [guide here](https://docs.unsloth.ai/basics/text-to-speech-tts-fine-tuning) **并查看我们的新** [Gemma 3N Guide](https://docs.unsloth.ai/basics/gemma-3n-how-to-run-and-fine-tune) ** Quants优先量优于其他量化方法！

请访问我们的所有文档，以获取我们的所有XX_MarkDown_link_xx和[Dynamic 2.0](https://docs.unsloth.ai/basics/unsloth-dynamic-2.0-ggufs)。


### 安装

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # 仅在COLAB笔记本上执行此操作！否则使用pip安装不塞
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth

### 不塞

“ FastModel”现在几乎支持加载任何模型！这包括视觉和文本模型！

In [ ]:
from unsloth import FastModel
import torch

fourbit_models = [
    "unsloth/Qwen3-4B-Instruct-2507-unsloth-bnb-4bit", # QWEN 14b 2x更快
    "unsloth/Qwen3-4B-Thinking-2507-unsloth-bnb-4bit",
    "unsloth/Qwen3-8B-unsloth-bnb-4bit",
    "unsloth/Qwen3-14B-unsloth-bnb-4bit",
    "unsloth/Qwen3-32B-unsloth-bnb-4bit",

    # 4位动态量子，以卓越的精度和低内存的使用
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/Phi-4",
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/orpheus-3b-0.1-ft-unsloth-bnb-4bit" # [新]我们支持TTS模型！
] # https://huggingface.co/unsloth的更多型号

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/Qwen3-4B-Instruct-2507",
    max_seq_length = 2048, # 为长篇小说选择任何！
    load_in_4bit = False,  # 4位量化以减少内存
    load_in_8bit = False, # [new！]更准确，使用2倍内存
    full_finetuning = False, # [new！]我们现在有完整的填充！
    # token =“ hf _...”，＃如果使用门控模型，请使用一个
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.2: Fast Qwen3 patching. Transformers: 4.54.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: QLoRA and full finetuning all not selected. Switching to 16bit LoRA.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.08G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

现在，我们添加Lora适配器，因此我们只需要更新少量参数！

In [ ]:
model = FastModel.get_peft_model(
    model,
    r = 32, # 选择任何数字> 0！建议8、16、32、64、128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0, # 支持任何人，但= 0已优化
    bias = "none",    # 支持任何人，但是=“无”
    # [NEW]“ Unsploth”使用的VRAM减少了30％，适合2倍批量的尺寸！
    use_gradient_checkpointing = "unsloth", # 在很长的背景下真实或“不整齐”
    random_state = 3407,
    use_rslora = False,  # 我们支持排名稳定的洛拉
    loftq_config = None, # 和Loftq
)

Unsloth: Making `model.base_model.model.model` require gradients


<a name =“ data”> </a>
###数据准备
现在，我们将“ QWEN-3”格式用于对话风格的Finetunes。我们在ShareGpt样式中使用XX_Markdown_link_xx数据集。 QWEN-3呈现多转向对话如下：

```
<|im_start|>user
Hello!<|im_end|>
<|im_start|>assistant
Hey there!<|im_end|>

```
我们使用“ get_chat_template”函数获取正确的聊天模板。我们支持`Zephyr，Chatml，Mistral，Llama，羊驼，Vicuna，Vicuna_old，Phi3，Llama3，Phi4，Qwen2.5，Gemma3`等。

In [ ]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen3-instruct",
)

In [ ]:
from datasets import load_dataset
dataset = load_dataset("mlabonne/FineTome-100k", split = "train")

README.md:   0%|          | 0.00/982 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/117M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

现在，我们使用`sandardize_data_formats`尝试将数据集转换为正确的格式，以进行列出！

In [ ]:
from unsloth.chat_templates import standardize_data_formats
dataset = standardize_data_formats(dataset)

Unsloth: Standardizing formats (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

让我们看看第100行的样子！

In [ ]:
dataset[100]

{'conversations': [{'content': 'What is the modulus operator in programming and how can I use it to calculate the modulus of two given numbers?',
   'role': 'user'},
  {'content': 'In programming, the modulus operator is represented by the \'%\' symbol. It calculates the remainder when one number is divided by another. To calculate the modulus of two given numbers, you can use the modulus operator in the following way:\n\n```python\n# Calculate the modulus\nModulus = a % b\n\nprint("Modulus of the given numbers is: ", Modulus)\n```\n\nIn this code snippet, the variables \'a\' and \'b\' represent the two given numbers for which you want to calculate the modulus. By using the modulus operator \'%\', we calculate the remainder when \'a\' is divided by \'b\'. The result is then stored in the variable \'Modulus\'. Finally, the modulus value is printed using the \'print\' statement.\n\nFor example, if \'a\' is 10 and \'b\' is 4, the modulus calculation would be 10 % 4, which equals 2. Theref

现在，我们必须将“ Qwen-3”的聊天模板应用于对话中，并将其保存到``text''中。

In [ ]:
def formatting_prompts_func(examples):
   convos = examples["conversations"]
   texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
   return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True)

Map:   0%|          | 0/100000 [00:00<?, ? examples/s]

让我们看看聊天模板是如何做到的！


In [ ]:
dataset[100]['text']

'<|im_start|>user\nWhat is the modulus operator in programming and how can I use it to calculate the modulus of two given numbers?<|im_end|>\n<|im_start|>assistant\nIn programming, the modulus operator is represented by the \'%\' symbol. It calculates the remainder when one number is divided by another. To calculate the modulus of two given numbers, you can use the modulus operator in the following way:\n\n```python\n# Calculate the modulus\nModulus = a % b\n\nprint("Modulus of the given numbers is: ", Modulus)\n```\n\nIn this code snippet, the variables \'a\' and \'b\' represent the two given numbers for which you want to calculate the modulus. By using the modulus operator \'%\', we calculate the remainder when \'a\' is divided by \'b\'. The result is then stored in the variable \'Modulus\'. Finally, the modulus value is printed using the \'print\' statement.\n\nFor example, if \'a\' is 10 and \'b\' is 4, the modulus calculation would be 10 % 4, which equals 2. Therefore, the output 

<a name =“ train”> </a>
###训练模型
现在，让我们使用huggingface trl的“ sfttrainer”！这里更多文档：[TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer)。我们执行60个步骤来加快速度，但是您可以将`num_train_epochs = 1`设置为完整运行，然后关闭`max_steps = none“。

In [ ]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # 可以设置评估！
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # 使用GA来模仿批处理大小！
        warmup_steps = 5,
        # num_train_epochs = 1，＃将其设置为1个完整的训练运行。
        max_steps = 60,
        learning_rate = 2e-4, # 长达2e-5用于长期训练
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # 将其用于Wandb等
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

我们还使用Unsploth的“ Train_on_completions”方法仅训练助手输出，而忽略用户输入的损失。这有助于提高Finetunes的准确性！

In [ ]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|im_start|>user\n",
    response_part = "<|im_start|>assistant\n",
)

Map (num_proc=2):   0%|          | 0/100000 [00:00<?, ? examples/s]

让我们验证掩盖说明部分完成了！让我们再次打印第100行。

In [ ]:
tokenizer.decode(trainer.train_dataset[100]["input_ids"])

'<|im_start|>user\nWhat is the modulus operator in programming and how can I use it to calculate the modulus of two given numbers?<|im_end|>\n<|im_start|>assistant\nIn programming, the modulus operator is represented by the \'%\' symbol. It calculates the remainder when one number is divided by another. To calculate the modulus of two given numbers, you can use the modulus operator in the following way:\n\n```python\n# Calculate the modulus\nModulus = a % b\n\nprint("Modulus of the given numbers is: ", Modulus)\n```\n\nIn this code snippet, the variables \'a\' and \'b\' represent the two given numbers for which you want to calculate the modulus. By using the modulus operator \'%\', we calculate the remainder when \'a\' is divided by \'b\'. The result is then stored in the variable \'Modulus\'. Finally, the modulus value is printed using the \'print\' statement.\n\nFor example, if \'a\' is 10 and \'b\' is 4, the modulus calculation would be 10 % 4, which equals 2. Therefore, the output 

现在，让我们打印出蒙面的例子 - 您应该只看到答案存在：

In [ ]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[100]["labels"]]).replace(tokenizer.pad_token, " ")

'                              In programming, the modulus operator is represented by the \'%\' symbol. It calculates the remainder when one number is divided by another. To calculate the modulus of two given numbers, you can use the modulus operator in the following way:\n\n```python\n# Calculate the modulus\nModulus = a % b\n\nprint("Modulus of the given numbers is: ", Modulus)\n```\n\nIn this code snippet, the variables \'a\' and \'b\' represent the two given numbers for which you want to calculate the modulus. By using the modulus operator \'%\', we calculate the remainder when \'a\' is divided by \'b\'. The result is then stored in the variable \'Modulus\'. Finally, the modulus value is printed using the \'print\' statement.\n\nFor example, if \'a\' is 10 and \'b\' is 4, the modulus calculation would be 10 % 4, which equals 2. Therefore, the output of the above code would be:\n\n```\nModulus of the given numbers is: 2\n```\n\nThis means that the modulus of 10 and 4 is 2.<|im_end|>

In [ ]:
# @Title显示当前内存统计
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

让我们训练模型吧！要恢复训练跑

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 100,000 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 66,060,288 of 4,088,528,384 (1.62% trained)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.259200
2,1.194400
3,1.737800
4,1.309600
5,0.903400
6,0.996900
7,0.530100
8,0.876800
9,0.702800
10,0.651600


In [ ]:
# @Title显示最终内存和时间统计
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

<a name =“推理”> </a>
###推理
让我们通过不舒服的本机推理来运行模型！根据“ QWEN-3”团队，建议推断的推荐设置为`= 0.7，top_p = 0.8，top_k = 20`

对于基于推理聊天的推断，`温度= 0.6，top_p = 0.95，top_k = 20`

In [ ]:
messages = [
    {"role" : "user", "content" : "Continue the sequence: 1, 1, 2, 3, 5, 8,"}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize = False,
    add_generation_prompt = True, # 必须添加一代
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer(text, return_tensors = "pt").to("cuda"),
    max_new_tokens = 1000, # 增加输出的增加！
    temperature = 0.7, top_p = 0.8, top_k = 20, # 对于非思考
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)

The sequence you provided is the Fibonacci sequence, where each number is the sum of the two preceding numbers. To continue the sequence, we add the last two numbers:

8 + 5 = 13

So, the next number in the sequence is 13.<|im_end|>


<a name =“ save”> </a>
###保存，加载固定模型
要将最终模型保存为Lora适配器，请使用HuggingFace的“ push_to_hub”进行在线保存或`save_pretaining'用于本地保存。

** [注意] **这仅保存洛拉适配器，而不是完整的模型。要节省16位或GGUF，请向下滚动！

In [ ]:
model.save_pretrained("lora_model")  # 本地节省
tokenizer.save_pretrained("lora_model")
# model.push_to_hub（“ your_name/lora_model”，token =“ ...”）＃在线保存
# tokenizer.push_to_hub（“ your_name/lora_model”，token =“ ...”）＃在线保存

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/chat_template.jinja',
 'lora_model/vocab.json',
 'lora_model/merges.txt',
 'lora_model/added_tokens.json',
 'lora_model/tokenizer.json')

现在，如果您想加载洛拉适配器，我们刚刚保存用于推理，请将`false``设置为true`：：true'：

In [ ]:
if False:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model", # 您用于培训的模型
        max_seq_length = 2048,
        load_in_4bit = True,
    )

### 节省float16 vllm

我们还直接支持保存到`float16`。为float16选择`merged_16bit`或用于int4的merged_4bit`。我们还允许`Lora`适配器作为后备。使用`push_to_hub_merged`上传到您的拥抱脸部帐户！您可以访问https://huggingface.co/settings/tokens for个人令牌。

In [ ]:
# 合并为16位
if False:
    model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: # 推向HF集线器
    model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# 合并到4位
if False:
    model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: # 推向HF集线器
    model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# 只是洛拉适配器
if False:
    model.save_pretrained("model")
    tokenizer.save_pretrained("model")
if False: # 推向HF集线器
    model.push_to_hub("hf/model", token = "")
    tokenizer.push_to_hub("hf/model", token = "")


### GGUF / LLAMA.CPP转换
要保存到`gguf' /`llama.cpp`，我们现在在本地支持它！我们克隆“ llama.cpp”，默认将其保存到Q8_0`。我们允许所有方法'q4_k_m`。使用`save_pretained_gguf`进行本地保存，然后`push_to_hub_gguf`将上传到HF。

一些支持的量化方法（我们的[Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)上的完整列表）：
*`Q8_0`-快速转换。大量资源使用，但通常可以接受。
*`q4_k_m`-推荐。使用q6_k作为注意到的一半。
*`q5_k_m`-推荐。使用q6_k进行一半的注意。wv和feed_forward.w2张量，else q5_k。

[** new **]要为Finetune和自动导出到Ollama，请尝试我们的[Ollama notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-olama.ipynb）

同样，如果您想将GGUF推到拥抱的脸部帐户上，请将“如果false”设置为“如果是true”，并添加拥抱的脸部令牌和上传位置！

In [ ]:
# 保存到8位Q8_0
if False:
    model.save_pretrained_gguf("model", tokenizer,)
# 请记住要访问https://huggingface.co/settings/tokens for代币！
# 并将HF更改为您的用户名！
if False:
    model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# 保存至16位GGUF
if False:
    model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: # 推向HF集线器
    model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# 保存到Q4_K_M GGUF
if False:
    model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: # 推向HF集线器
    model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# 保存到多个GGUF选项 - 如果要多个，请更快！
if False:
    model.push_to_hub_gguf(
        "hf/model", # 将HF更改为您的用户名！
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "", # 在https://huggingface.co/settings/tokens上获取一个令牌
    )

现在，在llama.cpp中使用`model-unsloth.gguf`文件或`model-unsloth-q4_k_m.gguf`文件或基于UI的系统（例如JAN或OPEN webUI）。您可以安装JAN XX_MARKDOWN_LINK_XX并打开webui [here](https://github.com/janhq/jan)

我们完成了！如果您对不舒服有任何疑问，我们有一个XX_MarkDown_link_xx频道！如果您发现任何错误或想对最新的LLM内容进行更新，或者需要帮助，加入项目等，请随时加入我们的不和谐！

其他一些链接：
1。训练您自己的推理模型-Llama grpo笔记本XX_Markdown_link_xx-grpo.ipynb）
2。将芬太尼保存到奥尔马。 [here](https://github.com/open-webui/open-webui)-ollama.ipynb）
3。Llama3.2视觉燃烧 -  X射线照相用例。 [Discord](https://discord.gg/unsloth)-vision.ipynb）
6。请参见DPO，ORPO，继续预处理，对话finetuning等笔记本，以及我们的xx_markDown_link_xx上的更多内容！

<div class =“ Align-Center”>
  <a href =“ https://unsloth.ai”> <img src =“ https://github.com/unslothai/unslothai/unsloth/raw/raw/main/main/images/unsloth%20new%20new%20logo.png”
  <a href =“ https://discord.gg/unsloth”> <img src =“ https://github.com/unslothai/unsloth/unsloth/raw/main/main/main/images/images/discord.png，png
  <a href =“ https://docs.unsloth.ai/”> <img src =“ https://github.com/unslothai/unslothai/unsloth/unsloth/main/main/main/images/images/images/documentation%20green%20green%20breen%20button.png?png?png?raw=true width =”

  如果您需要帮助，请加入DISCORD +⭐️<i>在<a href =“ https://github.com/unslothai/unsloth上”
</div>
